In [ ]:
import arcpy
arcpy.env.overwriteOutput = True

# -------------------
# USER INPUTS
# -------------------
workspace = r"C:\GIS\Scratch.gdb"

arcpy.env.workspace = workspace
nests    = "nogo_nest_usfs"
hab_high = "amgo_ecobject_cwhr_habmodel_high"
hab_mod  = "amgo_ecobject_cwhr_habmodel_moderate"

out_name = "TRPA_Goshawk_Threshold_Zone"
out_fc   = workspace + "\\" + out_name

target_total_acres = 500.0
buffer_dist = "0.25 Miles"

# Make “no holes” strict: remove ALL interior holes/parts by setting a huge threshold.
# Units are square units of the feature class coordinate system.
HOLE_AREA_THRESHOLD = 10**18

scratch = arcpy.env.scratchGDB  # disk-backed temp (reliable)

# -------------------
# HELPERS
# -------------------
def assert_exists(path, label):
    if not arcpy.Exists(path):
        raise RuntimeError(f"{label} was not created: {path}")

def add_acres(fc, field_name="Acres"):
    if field_name not in [f.name for f in arcpy.ListFields(fc)]:
        arcpy.management.AddField(fc, field_name, "DOUBLE")
    arcpy.management.CalculateGeometryAttributes(
        fc, [[field_name, "AREA_GEODESIC"]],
        area_unit="ACRES"
    )

def sum_field(fc, field_name):
    return sum((row[0] or 0.0) for row in arcpy.da.SearchCursor(fc, [field_name]))

def ensure_field(fc, name, ftype):
    if name not in [f.name for f in arcpy.ListFields(fc)]:
        arcpy.management.AddField(fc, name, ftype)

def cursor_order_by(fc, fields, order_by_field):
    # Best-effort ORDER BY (works in file gdb). Falls back if not supported.
    try:
        return arcpy.da.SearchCursor(fc, fields, sql_clause=(None, f"ORDER BY {order_by_field} ASC"))
    except Exception:
        return arcpy.da.SearchCursor(fc, fields)

def chunked_in_clause(field_name, values, chunk_size=900):
    values = list(values)
    for i in range(0, len(values), chunk_size):
        chunk = values[i:i+chunk_size]
        yield f"{field_name} IN ({','.join(map(str, chunk))})"

def copy_selected_oids(work_fc, oids, out_path):
    """Copy selected OIDs from work_fc to out_path."""
    if not oids:
        return None
    lyr = "sel_lyr"
    arcpy.management.MakeFeatureLayer(work_fc, lyr)
    arcpy.management.SelectLayerByAttribute(lyr, "CLEAR_SELECTION")

    oid_f = arcpy.Describe(work_fc).OIDFieldName
    for clause in chunked_in_clause(oid_f, oids):
        arcpy.management.SelectLayerByAttribute(lyr, "ADD_TO_SELECTION", clause)

    arcpy.management.CopyFeatures(lyr, out_path)
    assert_exists(out_path, "Selection copy")
    return out_path

def dissolve_parts(parts, out_path):
    arcpy.management.Merge([p for p in parts if p and arcpy.Exists(p)], out_path + "_m")
    assert_exists(out_path + "_m", "Merged")
    arcpy.management.Dissolve(out_path + "_m", out_path)
    assert_exists(out_path, "Dissolved")
    return out_path

def fill_holes(poly_fc, out_path):
    # Removes ALL interior holes/parts (and any tiny stray bits) by setting threshold huge.
    arcpy.management.EliminatePolygonPart(
        in_features=poly_fc,
        out_feature_class=out_path,
        condition="AREA",
        part_area=HOLE_AREA_THRESHOLD,
        part_option="ANY"
    )
    assert_exists(out_path, "Hole-filled polygon")
    return out_path

# -------------------
# OUTPUT FC (one polygon per nest)
# -------------------
sr = arcpy.Describe(hab_high).spatialReference
if arcpy.Exists(out_fc):
    arcpy.management.Delete(out_fc)

arcpy.management.CreateFeatureclass(workspace, out_name, "POLYGON", spatial_reference=sr)
for fn, ft in [
    ("NestOID", "LONG"),
    ("BufAc", "DOUBLE"),
    ("HighAc", "DOUBLE"),
    ("ModAc", "DOUBLE"),
    ("TotalAc", "DOUBLE"),
]:
    arcpy.management.AddField(out_fc, fn, ft)

# -------------------
# PER-NEST CONTIGUOUS GROWTH
# -------------------
nest_oid_field = arcpy.Describe(nests).OIDFieldName
nest_oids = [r[0] for r in arcpy.da.SearchCursor(nests, [nest_oid_field])]
print(f"Nests found: {len(nest_oids)}")

for idx, nest_oid in enumerate(nest_oids, start=1):
    print(f"\n[{idx}/{len(nest_oids)}] Nest OID {nest_oid}")

    # Single nest layer
    nest_lyr = "nest_lyr"
    arcpy.management.MakeFeatureLayer(nests, nest_lyr, f"{nest_oid_field} = {nest_oid}")

    # Required buffer
    buf_fc = scratch + fr"\buf_{nest_oid}"
    arcpy.analysis.Buffer(nest_lyr, buf_fc, buffer_dist, dissolve_option="ALL")
    assert_exists(buf_fc, "Buffer")

    add_acres(buf_fc, "Acres")
    buf_ac = sum_field(buf_fc, "Acres")
    need_out = target_total_acres - buf_ac

    # If buffer already >= 500, output buffer (filled)
    if need_out <= 0:
        final_fc = scratch + fr"\final_{nest_oid}"
        fill_holes(buf_fc, final_fc)
        add_acres(final_fc, "Acres")
        total_ac = sum_field(final_fc, "Acres")

        with arcpy.da.InsertCursor(out_fc, ["SHAPE@", "NestOID", "BufAc", "HighAc", "ModAc", "TotalAc"]) as ic:
            for (geom,) in arcpy.da.SearchCursor(final_fc, ["SHAPE@"]):
                ic.insertRow([geom, nest_oid, float(buf_ac), 0.0, 0.0, float(total_ac)])

        print(f"  Buffer-only Total: {total_ac:.2f} acres (>= 500)")
        continue

    # Erase buffer from habitat (prevents double counting where habitat overlaps buffer)
    high_out = scratch + fr"\high_out_{nest_oid}"
    mod_out  = scratch + fr"\mod_out_{nest_oid}"
    arcpy.analysis.Erase(hab_high, buf_fc, high_out)
    arcpy.analysis.Erase(hab_mod,  buf_fc, mod_out)
    assert_exists(high_out, "High outside")
    assert_exists(mod_out, "Mod outside")

    # Work copies that we can add fields to + compute Near once
    high_work = scratch + fr"\high_work_{nest_oid}"
    mod_work  = scratch + fr"\mod_work_{nest_oid}"
    arcpy.management.CopyFeatures(high_out, high_work)
    arcpy.management.CopyFeatures(mod_out,  mod_work)
    assert_exists(high_work, "High work")
    assert_exists(mod_work,  "Mod work")

    # Add Acres and NEAR_DIST (to the nest point) for compact growth ordering
    add_acres(high_work, "Acres")
    add_acres(mod_work,  "Acres")
    arcpy.analysis.Near(high_work, nest_lyr)  # adds NEAR_DIST
    arcpy.analysis.Near(mod_work,  nest_lyr)

    oid_high = arcpy.Describe(high_work).OIDFieldName
    oid_mod  = arcpy.Describe(mod_work).OIDFieldName

    # Layers for spatial selection
    high_lyr = "high_lyr"
    mod_lyr  = "mod_lyr"
    arcpy.management.MakeFeatureLayer(high_work, high_lyr)
    arcpy.management.MakeFeatureLayer(mod_work,  mod_lyr)

    # Maintain current contiguous footprint (starts at buffer)
    current_fc = scratch + fr"\current_{nest_oid}"
    arcpy.management.CopyFeatures(buf_fc, current_fc)

    selected_high_oids = set()
    selected_mod_oids  = set()
    high_added = 0.0
    mod_added  = 0.0

    # ---- Contiguous growth loop
    # Strategy:
    #   - Add HIGH polygons that touch the current footprint (contiguous), closest-to-nest first
    #   - If still short, add MOD polygons that touch the updated footprint, closest first
    #   - Update footprint by dissolving buffer + selected
    #
    # Note: This guarantees contiguity to the buffer. It won’t “jump” to isolated patches.
    #
    max_iters = 5000  # safety
    it = 0
    while (high_added + mod_added) < need_out and it < max_iters:
        it += 1
        made_progress = False

        # --- Try HIGH first
        if (high_added + mod_added) < need_out:
            arcpy.management.SelectLayerByAttribute(high_lyr, "CLEAR_SELECTION")
            arcpy.management.SelectLayerByLocation(
                in_layer=high_lyr,
                overlap_type="INTERSECT",
                select_features=current_fc,
                selection_type="NEW_SELECTION"
            )

            # Pull candidates in near order, add until we meet need or run out
            # Filter out already selected.
            where = None
            if selected_high_oids:
                # Exclude previously selected
                where = f"{oid_high} NOT IN ({','.join(map(str, sorted(selected_high_oids)))})"

            candidates = []
            if where:
                with cursor_order_by(high_work, [oid_high, "Acres", "NEAR_DIST"], "NEAR_DIST") as cur:
                    for oid, acres, nd in cur:
                        # Keep only those currently selected by location AND not selected before
                        # We’ll check membership via a quick selection layer test:
                        pass

                # Faster: use the selection set directly from the layer
                # and then sort by NEAR_DIST from the work FC.
            # Get selected OIDs from the layer
            sel_oids = set()
            with arcpy.da.SearchCursor(high_lyr, [oid_high]) as sc:
                for (o,) in sc:
                    if o not in selected_high_oids:
                        sel_oids.add(o)

            if sel_oids:
                # Sort selected candidates by NEAR_DIST asc
                rows = []
                where_sel = f"{oid_high} IN ({','.join(map(str, sorted(sel_oids)))})"
                with arcpy.da.SearchCursor(high_work, [oid_high, "Acres", "NEAR_DIST"], where_clause=where_sel) as cur:
                    for o, a, nd in cur:
                        rows.append((o, float(a or 0.0), float(nd if nd is not None else 1e18)))
                rows.sort(key=lambda x: x[2])

                for o, a, _ in rows:
                    if a <= 0:
                        continue
                    selected_high_oids.add(o)
                    high_added += a
                    made_progress = True
                    if (high_added + mod_added) >= need_out:
                        break

        # --- If still short, try MOD (must touch current footprint)
        if (high_added + mod_added) < need_out:
            arcpy.management.SelectLayerByAttribute(mod_lyr, "CLEAR_SELECTION")
            arcpy.management.SelectLayerByLocation(
                in_layer=mod_lyr,
                overlap_type="INTERSECT",
                select_features=current_fc,
                selection_type="NEW_SELECTION"
            )

            sel_oids = set()
            with arcpy.da.SearchCursor(mod_lyr, [oid_mod]) as sc:
                for (o,) in sc:
                    if o not in selected_mod_oids:
                        sel_oids.add(o)

            if sel_oids:
                rows = []
                where_sel = f"{oid_mod} IN ({','.join(map(str, sorted(sel_oids)))})"
                with arcpy.da.SearchCursor(mod_work, [oid_mod, "Acres", "NEAR_DIST"], where_clause=where_sel) as cur:
                    for o, a, nd in cur:
                        rows.append((o, float(a or 0.0), float(nd if nd is not None else 1e18)))
                rows.sort(key=lambda x: x[2])

                for o, a, _ in rows:
                    if a <= 0:
                        continue
                    selected_mod_oids.add(o)
                    mod_added += a
                    made_progress = True
                    if (high_added + mod_added) >= need_out:
                        break

        # If we couldn’t add anything contiguous, stop (no way to reach 500 contiguously)
        if not made_progress:
            break

        # Update current footprint (buffer + selected), dissolved
        parts = [buf_fc]
        if selected_high_oids:
            tmp_high_sel = scratch + fr"\tmp_high_sel_{nest_oid}"
            copy_selected_oids(high_work, sorted(selected_high_oids), tmp_high_sel)
            parts.append(tmp_high_sel)
        if selected_mod_oids:
            tmp_mod_sel = scratch + fr"\tmp_mod_sel_{nest_oid}"
            copy_selected_oids(mod_work, sorted(selected_mod_oids), tmp_mod_sel)
            parts.append(tmp_mod_sel)

        dissolve_parts(parts, current_fc)

    # Build final polygon from buffer + selected contiguous habitat
    parts = [buf_fc]
    if selected_high_oids:
        high_sel = scratch + fr"\high_sel_{nest_oid}"
        copy_selected_oids(high_work, sorted(selected_high_oids), high_sel)
        parts.append(high_sel)
    if selected_mod_oids:
        mod_sel = scratch + fr"\mod_sel_{nest_oid}"
        copy_selected_oids(mod_work, sorted(selected_mod_oids), mod_sel)
        parts.append(mod_sel)

    merged = scratch + fr"\merged_{nest_oid}"
    final_diss = scratch + fr"\final_diss_{nest_oid}"
    arcpy.management.Merge(parts, merged)
    arcpy.management.Dissolve(merged, final_diss)
    assert_exists(final_diss, "Final dissolved")

    # Fill holes (no interior rings)
    final_fc = scratch + fr"\final_noholes_{nest_oid}"
    fill_holes(final_diss, final_fc)

    add_acres(final_fc, "Acres")
    total_ac = sum_field(final_fc, "Acres")

    # Write to output
    with arcpy.da.InsertCursor(out_fc, ["SHAPE@", "NestOID", "BufAc", "HighAc", "ModAc", "TotalAc"]) as ic:
        for (geom,) in arcpy.da.SearchCursor(final_fc, ["SHAPE@"]):
            ic.insertRow([geom, nest_oid, float(buf_ac), float(high_added), float(mod_added), float(total_ac)])

    # Reporting
    if (high_added + mod_added) < need_out:
        print(f"  WARNING: Could not reach 500 acres contiguously.")
    print(f"  Buffer: {buf_ac:.2f} | High(add): {high_added:.2f} | Mod(add): {mod_added:.2f} | Total: {total_ac:.2f}")

print(f"\nDone. Output: {out_fc}")


Nests found: 136

[1/136] Nest OID 1
